In [ ]:
## NEED TO REWRITE THIS
# It's taking unique dates in the date_list for ALL games played, need to sort only by the games played by each team to account for off days

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [25]:
bat_data = pd.read_csv("Clean_Bat.csv",index_col = "Unnamed: 0")
bat_data.fillna(0,inplace=True)
bat_data.drop_duplicates(inplace=True)
pitch_data = pd.read_csv("Clean_Pitch.csv",index_col = "Unnamed: 0")
pitch_data.fillna(0,inplace=True)
pitch_data.drop_duplicates(inplace=True)
prior_games = pd.read_csv("Previous_Games.csv",index_col="Unnamed: 0")

In [26]:
total_sum_df = pd.DataFrame()
start_date = prior_games.iloc[0]['Date']
rolling_window_df = pd.DataFrame()
x = 0
while x < len(prior_games[prior_games['Date'] > start_date].Date.unique()):
    team_date_df = pd.DataFrame()
    date_list = prior_games[prior_games['Date'] > start_date].Date.unique()[x:x+7]
    for date in date_list:
        date_df = bat_data[bat_data['Date'] == date]
        team_df = pd.DataFrame()
        for team in date_df.Team.unique():
            temp_df = pd.DataFrame(date_df.groupby("Team").get_group(team)[['PA','AB','R','H','2B','3B','HR','RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']].sum()).T
            temp_df['Home_Away'] = int(date_df.Home_Away.iloc[0])
            temp_df['Date'] = date_df.Date.iloc[0]
            temp_df['Team'] = team
            temp_df.set_index("Team",inplace=True)
            team_df = pd.concat([team_df,temp_df])
        team_date_df = pd.concat([team_date_df,team_df])
    team_date_df.reset_index(inplace=True)
    for team in team_date_df.Team.unique():
        team_df = team_date_df
        team_df = team_df[team_df['Team'] == team]
        temp_df = pd.DataFrame(team_df.sum()[['PA','AB','R','H','2B','3B','HR','RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']]).T
        temp_df['Date'] = team_df.iloc[-1]['Date']
        temp_df['Team'] = team
        temp_df['Home_Away'] = team_df.iloc[-1].Home_Away
        total_sum_df = pd.concat([total_sum_df,temp_df])
    x+=1
if len(total_sum_df) > 0:
    final_df = pd.concat([rolling_window_df,total_sum_df])
    final_df.to_csv("7_Day_Window_Bat.csv")

In [34]:
total_sum_df = pd.DataFrame()
start_date = prior_games.iloc[0]['Date']
rolling_window_df = pd.DataFrame()
x = 0
while x < len(prior_games[prior_games['Date'] > start_date].Date.unique()):
    team_date_df = pd.DataFrame()
    date_list = prior_games[prior_games['Date'] > start_date].Date.unique()[x:x+7]
    for date in date_list:
        date_df = pitch_data[pitch_data['Date'] == date]
        team_df = pd.DataFrame()
        for team in date_df.Team.unique():
            temp_df = pd.DataFrame(date_df.groupby("Team").get_group(team)[['GS', 'IP', 'H', 'R', 'ER','BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB','CS', 'PO', 'BF', 'GB/FB', 'LD', 'PU']].sum()).T
            temp_df['Home_Away'] = int(date_df.Home_Away.iloc[0])
            temp_df['Date'] = date_df.Date.iloc[0]
            temp_df['Team'] = team
            temp_df.set_index("Team",inplace=True)
            team_df = pd.concat([team_df,temp_df])
        team_date_df = pd.concat([team_date_df,team_df])
    team_date_df.reset_index(inplace=True)
    for team in team_date_df.Team.unique():
        team_df = team_date_df
        team_df = team_df[team_df['Team'] == team]
        temp_df = pd.DataFrame(team_df.sum()[['GS', 'IP', 'H', 'R', 'ER','BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB','CS', 'PO', 'BF', 'GB/FB', 'LD', 'PU']]).T
        temp_df['Date'] = team_df.iloc[-1]['Date']
        temp_df['Team'] = team
        temp_df['Home_Away'] = team_df.iloc[-1].Home_Away
        total_sum_df = pd.concat([total_sum_df,temp_df])
    x+=1
if len(total_sum_df) > 0:
    final_df = pd.concat([rolling_window_df,total_sum_df])
    final_df.drop(columns = ['GB/FB','LD','PU'],inplace=True)
    final_df.to_csv("7_Day_Window_Pitch.csv")

2022-04-07
2022-04-08
2022-04-09
2022-04-10
2022-04-11
2022-04-12
2022-04-13
2022-04-14
2022-04-15
2022-04-16
2022-04-17
2022-04-18
2022-04-19
2022-04-20
2022-04-21
2022-04-22
2022-04-23
2022-04-24
2022-04-25
2022-04-26
2022-04-27
2022-04-28
2022-04-29
2022-04-30
2022-05-01
2022-05-02
2022-05-03
2022-05-04
2022-05-05
2022-05-06
2022-05-07
2022-05-08
2022-05-09
2022-05-10
2022-05-11
2022-05-12
2022-05-13
2022-05-14
2022-05-15
2022-05-16
2022-05-17
2022-05-18
2022-05-19
2022-05-20
2022-05-21
2022-05-22
2022-05-23
2022-05-24
2022-05-25
2022-05-26
2022-05-27
2022-05-28
2022-05-29
2022-05-30
2022-05-31
2022-06-01
2022-06-02
2022-06-03
2022-06-04
2022-06-05
2022-06-06
2022-06-07
2022-06-08
2022-06-09
2022-06-10
2022-06-11
2022-06-12
2022-06-13
2022-06-14
2022-06-15
2022-06-16
2022-06-17
2022-06-18
2022-06-19
2022-06-20
2022-06-21
2022-06-22
2022-06-23
2022-06-24
2022-06-25
2022-06-26
2022-06-27
2022-06-28
2022-06-29
2022-06-30
2022-07-01
2022-07-02
2022-07-03
2022-07-04
2022-07-05
2022-07-06

In [35]:
final_df[final_df['Date'] == "2023-06-12"]

,GS,IP,H,R,ER,BB,SO,HR,HBP,AB,...,IBB,GDP,SF,SB,CS,PO,BF,Date,Team,Home_Away
0,7.0,59.5,56.0,25.0,23.0,20.0,54.0,5.0,6.0,242.0,...,2.0,5.0,0.0,4.0,2.0,1.0,268.0,2023-06-12,SFN,0
0,6.0,49.7,44.0,32.0,29.0,19.0,43.0,7.0,1.0,194.0,...,0.0,4.0,4.0,3.0,0.0,0.0,222.0,2023-06-12,DET,0
0,7.0,64.6,57.0,27.0,25.0,29.0,73.0,6.0,5.0,246.0,...,1.0,5.0,1.0,4.0,2.0,2.0,281.0,2023-06-12,LAA,0
0,7.0,59.8,71.0,40.0,37.0,34.0,61.0,9.0,3.0,250.0,...,0.0,5.0,3.0,2.0,1.0,1.0,291.0,2023-06-12,COL,0
0,7.0,58.5,39.0,21.0,20.0,20.0,63.0,7.0,3.0,221.0,...,0.0,4.0,0.0,3.0,0.0,0.0,244.0,2023-06-12,TBA,0
0,7.0,61.3,63.0,37.0,34.0,20.0,61.0,12.0,1.0,247.0,...,0.0,7.0,2.0,4.0,1.0,0.0,271.0,2023-06-12,ATL,0
0,7.0,56.0,64.0,38.0,38.0,28.0,54.0,12.0,6.0,246.0,...,1.0,2.0,3.0,4.0,3.0,1.0,283.0,2023-06-12,CIN,0
0,6.0,47.9,57.0,37.0,36.0,19.0,41.0,8.0,0.0,198.0,...,0.0,3.0,3.0,6.0,1.0,1.0,221.0,2023-06-12,KCR,0
0,6.0,50.2,40.0,27.0,26.0,20.0,60.0,7.0,4.0,196.0,...,0.0,2.0,2.0,2.0,0.0,1.0,222.0,2023-06-12,PHI,0
0,6.0,50.1,48.0,33.0,30.0,21.0,48.0,11.0,3.0,205.0,...,0.0,6.0,1.0,6.0,2.0,0.0,230.0,2023-06-12,TEX,0
